In [2]:
import Bio.SeqIO as SeqIO
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def calculate_identity_from_aligned(seq1, seq2):
    """Calculates the percentage identity between two prealigned sequences."""
    if len(seq1) != len(seq2):
        raise ValueError("Aligned sequences must have the same length.")

    matches = sum(1 for a, b in zip(seq1, seq2) if a == b and a != '-' and b != '-')  # Ignore gaps
    total_positions = sum(1 for a, b in zip(seq1, seq2) if a != '-' and b != '-')  # Count non-gap positions

    if total_positions == 0:  # Avoid division by zero
        return 0.0

    identity = (matches / total_positions) * 100
    return identity

def create_combined_identity_matrix(aa_sequences, nt_sequences):
    """Creates a combined pairwise identity matrix using aligned amino acid and nucleotide sequences."""
    n = len(aa_sequences)
    if n != len(nt_sequences):
        raise ValueError("Amino acid and nucleotide sequence files must have the same number of sequences.")

    identity_matrix = np.zeros((n, n))  # Initialize matrix with zeros
    sequence_ids = [seq.id for seq in aa_sequences]  # Use IDs from amino acid sequences

    for i in range(n):
        for j in range(n):
            if i >= j:  # Below or on diagonal: calculate amino acid identity
                identity_matrix[i][j] = calculate_identity_from_aligned(str(aa_sequences[i].seq), str(aa_sequences[j].seq))
            else:  # Above diagonal: calculate nucleotide identity
                identity_matrix[i][j] = calculate_identity_from_aligned(str(nt_sequences[i].seq), str(nt_sequences[j].seq))

    return identity_matrix, sequence_ids

def visualize_heatmap(identity_matrix, sequence_ids, filename="S_combined_identity_heatmap.png"):
    """Visualizes the combined identity matrix as a heatmap with highlighted sequence names."""
    n = len(identity_matrix)
    
    # Create masks for amino acid and nucleotide parts
    mask_aa = np.triu(np.ones((n, n)), k=1)  # Mask for hiding upper part (nucleotides)
    mask_nt = np.tril(np.ones((n, n)), k=-1)  # Mask for hiding lower part (amino acids)

    df = pd.DataFrame(identity_matrix, index=sequence_ids, columns=sequence_ids)

    plt.figure(figsize=(12, 10))  # Adjust figure size
    
    # Heatmap for amino acids (Blues)
    sns.heatmap(df,
                annot=False,
                fmt=".1f",
                cmap="Blues",
                linewidths=0.5,
                mask=mask_aa,
                cbar_kws={'shrink': 0.8})  # Colorbar settings
    
    # Heatmap for nucleotides (Greens)
    sns.heatmap(df,
                annot=False,
                fmt=".1f",
                cmap="Greens",
                linewidths=0.5,
                mask=mask_nt,
                cbar_kws={'shrink': 0.8})  # Colorbar settings

    plt.title("S Segment Pairwise Identity Matrix (Amino Acids & Nucleotides)", fontsize=16)

    # Highlight first five sequence names in blue bold font
    ax = plt.gca()
    
    # Set x-tick labels with formatting
    x_labels = ax.get_xticklabels()
    y_labels = ax.get_yticklabels()
    
    for i in range(min(5, len(x_labels))):  # Highlight first five labels
        x_labels[i].set_fontsize(12)
        x_labels[i].set_weight('bold')
        x_labels[i].set_color('blue')
        
        y_labels[i].set_fontsize(12)
        y_labels[i].set_weight('bold')
        y_labels[i].set_color('blue')

    ax.set_xticklabels(x_labels)
    ax.set_yticklabels(y_labels)

    plt.xticks(rotation=90)  # Rotate x-axis labels for readability
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.savefig(filename)
    plt.show()
    print(f"Heatmap saved to {filename}")

# Main execution block
if __name__ == "__main__":
    aa_fasta_file = "allign_protein_S_Segm.fa"  # Replace your FASTA file 
    nt_fasta_file = "alligm_S_segm_nucleot.fa"  # replace your FASTA file


    try:
        aa_sequences = list(SeqIO.parse(aa_fasta_file, "fasta"))
        nt_sequences = list(SeqIO.parse(nt_fasta_file, "fasta"))

        if not aa_sequences or not nt_sequences:
            raise ValueError("One or both FASTA files are empty or contain no valid sequences.")

        identity_matrix, sequence_ids = create_combined_identity_matrix(aa_sequences, nt_sequences)
        visualize_heatmap(identity_matrix, sequence_ids)

    except FileNotFoundError:
        print(f"Error: One of the FASTA files '{aa_fasta_file}' or '{nt_fasta_file}' was not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


C:\Users\alexe\anaconda3\Lib\site-packages\seaborn\_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  from scipy.stats import gaussian_kde

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\alexe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\alexe\anaconda3\Lib\site-packages\traitlets\config\applicat

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\alexe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\alexe\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\alexe\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\alexe\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\alexe\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\alexe\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\alexe\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\alexe\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject